In [2]:
from __future__ import division
import tensorflow as tf
import numpy as np
import nltk
import pickle
import random
import argparse
import csv
import os
from numpy import dot
from numpy.linalg import norm
from gensim.models import FastText,Word2Vec
from gensim.models.phrases import Phrases, Phraser
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, strip_non_alphanum
import nltk
from nltk import sent_tokenize,word_tokenize
nltk.download('punkt')
import re
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.models import model_from_json


paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using TensorFlow backend.


In [4]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks')  
for file in os.listdir('.'):
  print(file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Untitled0.ipynb
Word2Vec.ipynb
vector_avg.ipynb
Doc2Vec.ipynb
news_cleaned_combined_ctext.txt
vector.pkl
newsCleaned_final.txt
news_cleaned_combined_ctext_final.txt
vector_summary.ipynb
text_token.txt
word2vec.model
newsCleaned_ctxt.txt
newsCleaned.txt
bbc_pkl
bbc_ft.bin
model.ipynb
Untitled1.ipynb
bbc_short_sum_pkl
models
vector_text.ipynb
model_bbc_short_summary.ipynb
Untitled2.ipynb
weights_satisfied.hdf5
outputs.npy
inputs.npy
model.json
model.h5
model_test.ipynb
model_bbc.ipynb


In [0]:
allWords = []
MAX_SENTENCE_LEN = 35
EMBEDDING_SIZE = 100
# p is the index into our training data for where we are now.
p = 0
accuracy_print_interval = 5
# number of neurons in each layer
input_num_units = MAX_SENTENCE_LEN*EMBEDDING_SIZE
hidden_num_units = 1500
output_num_units = MAX_SENTENCE_LEN
epochs = 250
batch_size = 32
learning_rate = 0.01

In [0]:
model_ft = FastText.load('bbc_ft.bin')

In [0]:
def clean_str(string):
    string = re.sub(r"\'s", "", string)
    string = re.sub(r"\'ve", "", string)
    string = re.sub(r"n\'t", "", string)
    string = re.sub(r"\'re", "", string)
    string = re.sub(r"\'d", "", string)
    string = re.sub(r"\'ll", "", string)
    string = re.sub(r",", "", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", "", string)
    string = re.sub(r"\)", "", string)
    string = re.sub(r"\?", "", string)
    string = re.sub(r"'", "", string)
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"[0-9]\w+|[0-9]","", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"\n",'',string)
    string = re.sub(r"  "," ",string)
    return string.strip().lower()

In [0]:
def vectorize_texts_and_summaries(texts, summaries):
    ct=0
    global MAX_SENTENCE_LEN
    nested_list_len = lambda x: sum(len(list) for list in x)
    t_vectors = []
    source_text_vectors = np.zeros((nested_list_len(texts), MAX_SENTENCE_LEN*EMBEDDING_SIZE))
    s_vectors = []
    target_summary_vectors = np.zeros((nested_list_len(texts), MAX_SENTENCE_LEN))
    vec_idx = 0
    not_vocab=[]
    if(type(texts[0]) == list):
        for i in range(len(texts)):
            summary = summaries[i]
            sentences = texts[i]
            sentences_container = []
            # Get text vector
            for s in sentences:
                sentence_vector = np.array([])
                target_vector = np.array([])
                s=remove_stopwords(strip_punctuation(strip_non_alphanum(str(s).lower())))
                s=clean_str(s)
                for w in word_tokenize(s):
                    if(model_ft.__contains__(w)==False):
                      print(w)
                      continue
                    if(len(sentence_vector) < MAX_SENTENCE_LEN*EMBEDDING_SIZE):
                        sentence_vector = np.append(sentence_vector, model_ft[w])
                        target_vector = np.append(target_vector, int(w in summary))
                    else:
                        break
                while(len(sentence_vector) < MAX_SENTENCE_LEN*EMBEDDING_SIZE):
                    sentence_vector = np.append(sentence_vector,np.zeros(EMBEDDING_SIZE))
                    target_vector = np.append(target_vector, 0)
                source_text_vectors[vec_idx] = sentence_vector
                target_summary_vectors[vec_idx] = target_vector
                vec_idx+=1
    return (source_text_vectors, target_summary_vectors)

In [0]:
import pickle
with open('bbc_pkl/texts.pkl', 'rb') as pickle_file:
    source=pickle.load(pickle_file)
with open('bbc_pkl/summary.pkl', 'rb') as pickle_file:
    target=pickle.load(pickle_file)
    
print(len(source))
print(len(target))
print(source[0])
print(target[0])

2225
2225
['quarterly profits at us media giant timewarner jumped to for the three months to december from year earlier', 'the firm which is now one of the biggest investors in google benefited from sales of high speed internet connections and higher advert sales', 'timewarner said fourth quarter sales rose to from', 'its profits were buoyed by one off gains which offset a profit dip at warner bros and less users for aol', 'time warner said on friday that it now owns of search engine google', 'but its own internet business aol had has mixed fortunes', 'it lost subscribers in the fourth quarter profits were lower than in the preceding three quarters', 'however the company said aol underlying profit before exceptional items rose on the back of stronger internet advertising revenues', 'it hopes to increase subscribers by offering the online service free to timewarner internet customers and will try to sign up aol existing customers for high speed broadband', 'timewarner also has to restat

In [0]:
source_vec,target_vec=vectorize_texts_and_summaries(source,target)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


rj
xlr
dz
dhl
rj
cfd
cfd
cfd
pwc
ccf
kk
hk
hk
zdf
rj
rj
jd
jd
lr
kk
pwc
pwc
pwc
jm
jm
jfk
dmx
pj
vs
jj
nx
rdf
rdf
nrj
nrj
vp
pj
pj
xxl
jm
jm
cfx
cfx
bwv
vs
hq
hq
www
ldv
fh
kp
lw
kp
lw
kp
kp
qv
ql
www
yh
jf
vj
vj
hl
hl
sd
sd
sd
wcg
wcg
cvs
jvc
jvc
cvs
sd
www
yh
gfk
vj
qv
ql
wltm
tdg
lzw


In [0]:
# SAVE
np.save('inputs.npy', source_vec, allow_pickle=False)
np.save('outputs.npy', target_vec, allow_pickle=False)
# LOAD
# source_vec = np.load('inputs.npy')
# target_vec = np.load('outputs.npy')
print(len(source_vec),len(target_vec))

41403 41403


In [0]:
model = Sequential()
model.add(Dense(1500, input_dim=3500, activation='relu'))
model.add(Dense(35, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1500)              5251500   
_________________________________________________________________
dense_2 (Dense)              (None, 35)                52535     
Total params: 5,304,035
Trainable params: 5,304,035
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(source_vec, target_vec, epochs=100, batch_size=32,verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
41403/41403 [==============================] - 15s 353us/step - loss: 15.8458
Epoch 2/100
41403/41403 [==============================] - 13s 321us/step - loss: 15.6566
Epoch 3/100
41403/41403 [==============================] - 14s 331us/step - loss: 15.6183
Epoch 4/100
41403/41403 [==============================] - 13s 323us/step - loss: 15.5689
Epoch 5/100
41403/41403 [==============================] - 13s 321us/step - loss: 15.4926
Epoch 6/100
41403/41403 [==============================] - 13s 318us/step - loss: 15.3985
Epoch 7/100
41403/41403 [==============================] - 13s 324us/step - loss: 15.2981
Epoch 8/100
41403/41403 [==============================] - 13s 323us/step - loss: 15.1981
Epoch 9/100
41403/41403 [==============================] - 13s 318us/step - loss: 15.1116
Epoch 10/100
41403/41403 [==============================] - 13s 317us/step - loss: 15.0341
Epoch 11/100
41403/41403 [=========================

In [0]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
# print(source_vec[32221])
# print(target_vec[36])
# text=str(source[0])+'.'+str(source[1])
print(source_vec.shape)
try_m=source_vec[213:244]
pred=model.predict(try_m)
print(pred)
# print(pred[224])
# print(source_vec[12])
# print(model.predict(source_vec[0]))
# print(len(source[0]))
# for s in source[2]:
#   vec=vectorize_text(s)
#   print(vec.shape)
#   pred=model.predict(vec)
#   print(pred)
# ere=vectorize_text(source[2])
# print(ere.shape)

(41403, 3500)
[[9.19788554e-02 9.16500241e-02 9.10192728e-02 ... 1.26146143e-14
  1.20307300e-14 1.20752373e-14]
 [1.25485181e-03 7.37651271e-06 2.88824467e-05 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.45009363e-01 1.18840169e-02 2.26092964e-01 ... 1.89275711e-25
  1.11128611e-25 1.19270912e-25]
 ...
 [4.28532576e-03 8.23761025e-12 1.24379555e-02 ... 2.97507001e-13
  2.15142601e-13 1.60551178e-13]
 [1.25053212e-01 1.25871778e-01 1.23451665e-01 ... 7.42070428e-18
  6.93971860e-18 7.72547070e-18]
 [8.90662894e-02 8.54089707e-02 8.46719071e-02 ... 2.31792485e-11
  2.18902362e-11 2.10581413e-11]]


In [0]:
print(predictions[10])
text=source[0]
def test(text):
  text=str(text)
  text_vec=vectorize_text(text)
  results=model.predict(text_vec)
  word_idx=sorted(range(len(results)), key=lambda i: lst[i], reverse=True)[:15]
  for i in word_idx:
    results[i]=1
  for res in results:
      print('##')
      print( " ".join([word for idx,word in enumerate(text.split(" ")) if(res[idx] == 1)]) )
  


[9.2768952e-02 9.2036739e-02 8.6075410e-02 7.2006881e-02 8.3035894e-02
 8.1244640e-02 8.1764631e-02 8.2747318e-02 8.5876934e-02 7.8830443e-02
 7.6566115e-02 8.7046020e-02 8.6242062e-09 3.1908178e-14 3.3938084e-15
 6.6952478e-15 1.1516922e-14 1.8075825e-14 1.4731428e-15 2.8180346e-15
 3.5107614e-15 3.2163021e-15 2.9392710e-15 2.5641419e-15 2.9177212e-15
 3.0298735e-15 2.8936354e-15 3.2966023e-15 4.2572439e-15 4.2644285e-15
 3.4237747e-15 4.1406664e-15 4.4241238e-15 4.2321222e-15 3.5479946e-15]
